In [10]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
 
n_qubits = 75
 
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=n_qubits)
 
rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
 
pubs = []
circuits = [IQP(mat) for mat in mats]
observables = [
    SparsePauliOp("Z" * n_qubits),
    SparsePauliOp("Z" * n_qubits),
    SparsePauliOp("Z" * n_qubits),
    SparsePauliOp("Z" * n_qubits),
    SparsePauliOp("Z" * n_qubits),
    SparsePauliOp("Z" * n_qubits),
]
 
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))
 
estimator = Estimator(backend)

job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: 0.08333333333333333
>>> Standard errors for PUB 0: 0.33205990027081356
>>> Expectation values for PUB 1: -0.38235294117647056
>>> Standard errors for PUB 1: 0.48639575633760457
>>> Expectation values for PUB 2: 0.3541666666666667
>>> Standard errors for PUB 2: 0.7774816130841108


In [9]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='63daf9a7e4a8d389f25342b84a4592fe47bba144a477be3383a3f7759c1c32d6e80ba0e04899b0949b0ac9821c45ba35f29c704e9a090f7ba4a938b59ede3785'
)
job = service.job('ctn0rrb907y0008w8hrg')
job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")

Expectation values for pub 0: -0.2857142857142857
Expectation values for pub 1: -0.01904296875
Expectation values for pub 2: 0.18333333333333332
